<a href="https://colab.research.google.com/github/Beloveddream/RS/blob/main/movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install deepctr

     |████████████████████████████████| 133kB 8.5MB/s 
     |████████████████████████████████| 2.9MB 14.3MB/s 
ERROR: tensorflow 2.5.0 has requirement h5py~=3.1.0, but you'll have h5py 2.10.0 which is incompatible.
  Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0


In [6]:
pip install surprise

     |████████████████████████████████| 11.8MB 224kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617622 sha256=588523ecae72386815e4872d26638f9427e561388c8ed16ec6fe97eb3e9a2e09
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from deepctr.models import DeepFM
#from deepctr.inputs import SparseFeat,get_feature_names
from deepctr.feature_column import SparseFeat,get_feature_names

**DeepFM**

In [17]:
#数据加载
data = pd.read_csv("/content/gdrive/My Drive/movielens/movielens_sample.txt")
sparse_features = ["movie_id", "user_id", "gender", "age", "occupation", "zip"]
target = ['rating']

# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
print(fixlen_feature_columns)
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

# 使用DeepFM进行训练
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2, )
# 使用DeepFM进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

[SparseFeat(name='movie_id', vocabulary_size=187, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.initializers_v1.RandomNormal object at 0x7f1a5107c090>, embedding_name='movie_id', group_name='default_group', trainable=True), SparseFeat(name='user_id', vocabulary_size=193, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.initializers_v1.RandomNormal object at 0x7f1a5053e2d0>, embedding_name='user_id', group_name='default_group', trainable=True), SparseFeat(name='gender', vocabulary_size=2, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.initializers_v1.RandomNormal object at 0x7f1a50541e50>, embedding_name='gender', group_name='default_group', trainable=True), SparseFeat(name='age', vocabulary_size=7, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializer

**Wide_Deep** 

In [15]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr.models import WDL
#from deepctr.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr.feature_column import SparseFeat,get_feature_names,DenseFeat

In [18]:
data = pd.read_csv('/content/gdrive/My Drive/movielens/movielens_sample.txt')
sparse_features = ['user_id', 'movie_id', 'timestamp', 'title', 'genres', 'gender', 'occupation', 'zip']
dense_features = ['age']
target = ['rating']

# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])

mms = MinMaxScaler(feature_range=(0,1))
data[dense_features] = mms.fit_transform(data[dense_features])

# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique())
                       for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1)
                      for feat in dense_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

# 使用WDL进行训练
model = WDL(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'] )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=10, verbose=True, validation_split=0.2 )
# 使用WDL进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 15.0138 - mse: 15.0138 - val_loss: 15.6914 - val_mse: 15.6913
Epoch 2/10
1/1 [==============================] - 0s 25ms/step - loss: 14.6977 - mse: 14.6977 - val_loss: 15.4098 - val_mse: 15.4098
Epoch 3/10
1/1 [==============================] - 0s 24ms/step - loss: 14.3825 - mse: 14.3825 - val_loss: 15.1310 - val_mse: 15.1310
Epoch 4/10
1/1 [==============================] - 0s 24ms/step - loss: 14.0714 - mse: 14.0714 - val_loss: 14.8550 - val_mse: 14.8550
Epoch 5/10
1/1 [==============================] - 0s 27ms/step - loss: 13.7645 - mse: 13.7645 - val_loss: 14.5798 - val_mse: 14.5798
Epoch 6/10
1/1 [==============================] - 0s 33ms/step - loss: 13.4575 - mse: 13.4574 - val_loss: 14.3024 - val_mse: 14.3024
Epoch 7/10
1/1 [==============================] - 0s 46ms/step - loss: 13.1481 - mse: 13.1481 - val_loss: 14.0174 - val_mse: 14.0174
Epoch 8/10
1/1 [==============================] - 0s 36ms/step - loss: 

**SVD**

In [7]:
from surprise import SVD , SVDpp
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import KFold
import pandas as pd
import time

import os

In [9]:
os.chdir("/content/gdrive/My Drive/les2")
time1=time.time()

# 数据读取
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
data = Dataset.load_from_file('ratings.csv', reader=reader)
train_set = data.build_full_trainset()

In [10]:
# 使用funkSVD
algo_1 = SVD(biased=False)

# 定义K折交叉验证迭代器，K=3
kf_1 = KFold(n_splits=3)
print("使用funkSVD模型：")
for trainset, testset in kf_1.split(data):
    # 训练并预测
    algo_1.fit(trainset)
    predictions = algo_1.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

使用funkSVD模型：
RMSE: 0.8758
RMSE: 0.8726
RMSE: 0.8723


In [11]:
# 使用BiasSVD
algo_2 = SVD(biased=True)

# 定义K折交叉验证迭代器，K=3
kf_2 = KFold(n_splits=3)
print("使用BiasSVD模型：")
for trainset, testset in kf_2.split(data):
    # 训练并预测
    algo_2.fit(trainset)
    predictions = algo_2.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

# 使用SVD++
algo_3 = SVDpp()

# 定义K折交叉验证迭代器，K=3
kf_3 = KFold(n_splits=3)
print("使用SVD++模型：")
for trainset, testset in kf_3.split(data):
    # 训练并预测
    algo_3.fit(trainset)
    predictions = algo_3.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

time2=time.time()
print(time2-time1)

使用BiasSVD模型：
RMSE: 0.8431
RMSE: 0.8466
RMSE: 0.8449
使用SVD++模型：


KeyboardInterrupt: ignored